In [1]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from lib.BackTest import BackTest
from lib.models import BaseModel
from prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from prebuilt.riskManagement import TakeProfitStopLoss
from custom.equitiesSelection import SpacsSelectionModel
from custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        # self.risk_control = TakeProfitStopLoss(100, 20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.008585943281872288
0.005831881383802343


In [8]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TakeProfitStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Stop Loss 10 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TakeProfitStopLoss(100, 10)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed

1 Failed download:
['AGBA']: YFPricesMissingError('$%ticker%: possibly delisted; no price data found  (1d 2022-08-07 -> 2024-08-07)')


KeyboardInterrupt: 

In [9]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 20 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.006943175014047314
0.00461451042038434


In [10]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 10 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(10)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.004870736158417341
0.0026952027738360914


In [11]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.hedgingSystem import BaseHedgeFeature
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 20 with Hedging Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(20)
        self.hedging_system = BaseHedgeFeature('QQQ')

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.0044263964511997
0.003001310229830933


In [12]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.hedgingSystem import BaseHedgeFeature
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with Trailling Stop Loss 10 with Hedging Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 10)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(10)
        self.hedging_system = BaseHedgeFeature('QQQ')

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.0028947201284282883
0.0016274554473900324


In [3]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.prebuilt.riskManagement import TraillingStopLoss
from server.custom.equitiesSelection import SpacsSelectionModel
from server.custom.strategies import SpacSignals

class SpacInsiderLogisticRegressionModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'SPAC Insider Logistic Regression with 20% Allocation with Trailling Stop Loss 20 Model'
        self.equity_selection_management = SpacsSelectionModel(40, 5, 'Short', self.start_date, self.end_date)
        self.portfolio_management = PercentagePortfolioAllocation(1000000, 20)
        self.alpha_model = SpacSignals()
        self.risk_control = TraillingStopLoss(20)

Test = BackTest(SpacInsiderLogisticRegressionModel)
Test.run()

[*********************100%***********************]  70 of 70 completed


0.011042316563898659
0.007373908195456366


In [1]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel

class BaseHoldNVDAModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'Base Hold Model'

Test = BackTest(BaseHoldNVDAModel)
Test.run()

[*********************100%***********************]  2 of 2 completed
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.tradelog = pd.concat([self.tradelog, row], ignore_index=True)
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:339: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.portfolio_records = pd.concat([self.portfolio_records, pd.DataFrame([self.portfolio_log])], ignore_index=True)


0.010287553956026579
0.007013319470218266


/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:104: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trading_records = pd.concat([self.trading_records, log], ignore_index=True)


In [1]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.strategies import Double7Signal

class BaseHoldNVDAModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'Double 7 Signals Alpha'
        self.equities = ['PLTR']
        self.alpha_model = Double7Signal()

Test = BackTest(BaseHoldNVDAModel)
Test.run()

[*********************100%***********************]  2 of 2 completed
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:339: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.portfolio_records = pd.concat([self.portfolio_records, pd.DataFrame([self.portfolio_log])], ignore_index=True)
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.tradelog = pd.concat([self.tradelog, row], ignore_index=True)
/home/duncan/Docume

0.02666369618056481
0.013982219828057115


/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:260: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  stonk = self.portfolio_records['Total'].pct_change()


In [3]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.strategies import Double7Signal
from server.prebuilt.riskManagement import TraillingStopLoss

class BaseHoldNVDAModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'Base Hold PLTR Model'
        self.equities = ['PLTR']

        self.alpha_model = Double7Signal()

Test = BackTest(BaseHoldNVDAModel)
Test.run()

[*********************100%***********************]  2 of 2 completed
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.tradelog = pd.concat([self.tradelog, row], ignore_index=True)
/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:339: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.portfolio_records = pd.concat([self.portfolio_records, pd.DataFrame([self.portfolio_log])], ignore_index=True)


0.04130036079132279
0.023783512100738098


/home/duncan/Documents/github/AlphaTrade/server/lib/BackTest.py:104: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  self.trading_records = pd.concat([self.trading_records, log], ignore_index=True)


In [13]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.strategies import Double7Signal
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class Double7SignalModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2022-08-07"
        self.end_date = "2024-10-17"
        self.model_name = 'Double 7 Signal with Tech Stocks with Trailling Stop Loss 10 Model'
        self.equities = ['TTEK']
        self.alpha_model = Double7Signal()
        self.risk_control = TraillingStopLoss(5)

Test = BackTest(Double7SignalModel)
Test.run()

[*********************100%***********************]  2 of 2 completed


Triggered 2023-08-10 00:00:00+00:00
Triggered 2023-10-24 00:00:00+00:00
Triggered 2024-06-03 00:00:00+00:00
0.007447899701119547
0.0049395364492153164


In [9]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.strategies import Double7Signal
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class Double7SignalModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2021-08-07"
        self.end_date = "2024-08-07"
        self.model_name = 'Double 7 Signal with Trailling Stop Loss 10 Model'
        self.equities = ['PLTR']
        self.alpha_model = Double7Signal()
        self.risk_control = TraillingStopLoss(20)

Test = BackTest(Double7SignalModel)
Test.run()

[*********************100%***********************]  2 of 2 completed


Triggered 2023-08-17 00:00:00+00:00
Triggered 2024-08-05 00:00:00+00:00
0.021233351167644517
0.010674375279961467


In [1]:
from server.lib.BackTest import BackTest
from server.lib.models import BaseModel
from server.prebuilt.strategies import Double7Signal
from server.prebuilt.riskManagement import TraillingStopLoss
from server.prebuilt.portfolioAllocations import PercentagePortfolioAllocation
from server.lib.universe import BaseUniverseSetting

import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

class Double7SignalModel(BaseModel):
    def __init__(self):
        super().__init__()
        self.start_date = "2024-09-21"
        self.end_date = "2024-10-21"
        self.universe_management = BaseUniverseSetting()
        self.universe_management.setIntervals(['5m'])
        self.model_name = 'ETH-USD Double 7 Signal Model'
        self.equities = ['ETH-USD']
        self.alpha_model = Double7Signal()

Test = BackTest(Double7SignalModel)
Test.run()

[*********************100%***********************]  2 of 2 completed


0.0006740764404481221
0.0004805286929090819
